In [100]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)

df = pd.read_csv('C3_anonymized.csv')
df.head(2)

,article_id,comment_author,comment_counter,comment_text,njudgements_constructiveness_expt,njudgements_toxicity_expt,agree_constructiveness_expt,agree_toxicity_expt,constructive,crowd_toxicity_level,...,constructive_characteristics,non_constructive_characteristics,toxicity_characteristics,crowd_comments_constructiveness_expt,crowd_comments_toxicity_expt,other_con_chars,other_noncon_chars,other_toxic_chars,constructive_binary,pp_comment_text
0,26023945,0,source1_26023945_62,And this Conservative strategy has produced th...,3.0,3.0,0.17,0.50,1.0,4.0,...,specific_points:3\ndialogue:2,no_non_con:3\nprovocative:1,abusive:3\npersonal_attack:1\nteasing:1\nembar...,\n\n,\n\n,\n\n,\n\n,\n\n,1.0,And this Conservative strategy has produced th...
1,24565777,1,source1_24565777_106,I commend Harper for holding the debates outsi...,3.0,3.0,0.33,0.17,1.0,3.0,...,specific_points:3\ndialogue:2,no_non_con:2\nno_respect:1,abusive:1\npersonal_attack:1\nteasing:1\nembar...,\n\n,\n\n,\n\n,\n\n,\n\n,1.0,I commend Harper for holding the debates outsi...


In [102]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]

    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    processed_text = ' '.join(lemmatized_tokens)

    return processed_text

df['processed_text'] = df['comment_text'].apply(preprocess_text)


In [103]:
analyzer = SentimentIntensityAnalyzer()

def analise2(score):
  if score > 0:
    return 'positive'
 
  return 'negative'
    
df['sentiment'] = df['comment_text'].apply(get_sentiment)

In [104]:
tfidf = TfidfVectorizer(max_features=50) 
cat_tfidf = tfidf.fit_transform(df['comment_text'].fillna('')).toarray()

cat_df = pd.DataFrame(cat_tfidf, columns=[f'cat_{i}' for i in range(cat_tfidf.shape[1])])
df = pd.concat([df.reset_index(drop=True), cat_df], axis=1)

In [138]:
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
df = df.replace('', np.nan)
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
X = df.drop(['agree_constructiveness_expt', 'njudgements_constructiveness_expt', 'constructive', 'other_toxic_chars', 'other_noncon_chars', 'pp_comment_text', 'other_con_chars', 'crowd_comments_toxicity_expt', 'processed_text', 'constructive_binary', 'comment_counter', 'article_id', 'pp_comment_text', 'comment_text', 'has_content', 'crowd_discard', 'constructive_characteristics', 'non_constructive_characteristics', 'toxicity_characteristics', 'crowd_comments_constructiveness_expt'], axis=1)
y = df['constructive_binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("\nConfusion Matrix:\n", conf_matrix)

# Optional: Detailed report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.895
Precision: 0.9007874015748032
Recall: 0.9007874015748032
F1 Score: 0.9007874015748032

Confusion Matrix:
 [[1004  126]
 [ 126 1144]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.89      0.89      0.89      1130
         1.0       0.90      0.90      0.90      1270

    accuracy                           0.90      2400
   macro avg       0.89      0.89      0.89      2400
weighted avg       0.90      0.90      0.90      2400

